## Data enhancement

In [11]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder

### Pre-processing images in data
** note that only training data will undergo enhancement

#### Image enhancement
The process of improving the appearance of images to highlight specific features, reduce noise or improve the quality of the image, this helps it to be more suitable for analysis.

Alpha focuses on the contrast of image
- alpha greater than 1: images brighter, enhanced contrast
- alpha less than 1: images darker, reduced contrast

Beta focuses on brightness of image
- beta positive: makes images brighter
- beta negative: makes images darker

In [2]:
train_data = pd.read_csv("datasets/csv/train_data.csv")
test_data = pd.read_csv("datasets/csv/test_data.csv")

## PREPROCESSING IMAGES WITH ENHANCEMENT
def preprocess_image(img_path, enhance):
    # reading images
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if enhance == True:
        # enhancing image
        img = cv2.convertScaleAbs(img, alpha = 1.5, beta = -20)
    # target_size of 224, 224 commonly used for image classification
    img = cv2.resize(img, (224, 224))
    # normalising pixel values
    img_array = img.astype(np.float32) / 255
    return img_array

In [3]:
# using preprocessed images as train data
train_images = np.array([preprocess_image(image_path, True) for image_path in train_data["image_path"]])
# using "pathology" column as train labels
train_labels = np.array(train_data["pathology"])

# change "BENIGN_WITHOUT_CALLBACK" to "BENIGN"
train_labels[train_labels == "BENIGN_WITHOUT_CALLBACK"] = "BENIGN"

In [4]:
# using preprocessed images as test data
test_images = np.array([preprocess_image(image_path, False) for image_path in test_data["image_path"]])
# using "pathology" column as test labels
test_labels = np.array(test_data["pathology"])

# change "BENIGN_WITHOUT_CALLBACK" to "BENIGN"
test_labels[test_labels == "BENIGN_WITHOUT_CALLBACK"] = "BENIGN"

#### Image augmentation

In [5]:
## IMAGE AUGMENTATION - using numpy
augmented_images = []
augmented_labels = []

# augmenting images and storing in lists
for i, img_path in enumerate(train_data["image_path"]):
    original_image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    enhanced_image = preprocess_image(img_path, True)
    
    # adding original image and label
    augmented_images.append(enhanced_image)
    augmented_labels.append(train_labels[i])
    
    # making all combinations of flips
    for horizontal_flip in [True, False]:
        for vertical_flip in [True, False]:
            # applying flips
            augmented_image = enhanced_image
            if horizontal_flip:
                augmented_image = np.fliplr(augmented_image)
            if vertical_flip:
                augmented_image = np.flipud(augmented_image)
                
            # adding augmented image and label
            augmented_images.append(augmented_image)
            augmented_labels.append(train_labels[i])
        
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

In [6]:
# encoding labels
label_encoder = LabelEncoder()
encoded_train_labels = label_encoder.fit_transform(train_labels)
encoded_aug_train_labels = label_encoder.fit_transform(augmented_labels)
encoded_test_labels = label_encoder.fit_transform(test_labels)
# one-hot encode labels
one_hot_train_labels = tf.keras.utils.to_categorical(encoded_train_labels)
one_hot_aug_train_labels = tf.keras.utils.to_categorical(encoded_aug_train_labels)
one_hot_test_labels = tf.keras.utils.to_categorical(encoded_test_labels)

In [7]:
## ## VERSION 2 OF IMAGE AUGMENTATION - using TensorFlow
## datagen = ImageDataGenerator(
##     horizontal_flip = True,
##     vertical_flip = True,
##     fill_mode = "nearest"
## )
## 
## augmented_generator = datagen.flow(train_images[:, :, :, np.newaxis], one_hot_train_labels, batch_size = 32)

## Creating of models
- **Conv2D** performs convolutional operations on the input image data. It applies a set of filters to the input images to extract features.
- **MaxPooling2D** is a down-sampling operation that reduces the spatial dimensions, used after Conv2D layers to retain the most important information.
- **Flatten** is used to convert the multi-dimensional output of the previous laters into 1D.
- **Dense** represents a fully connected layer, where each neuron or node is connected to every neuron in the previous layer.

### Base model
Ensures that the data can be trained on.

In [8]:
base_model = Sequential()
# creating stack of Conv2D and MaxPooling2D
base_model.add(Conv2D(32, (3, 3), activation = "relu", input_shape = (224, 224, 1)))
base_model.add(MaxPooling2D((2, 2)))

# unrolling output to 1D
base_model.add(Flatten())
base_model.add(Dense(128, activation = "relu"))
# output layer with softmax
base_model.add(Dense(2, activation = "softmax"))

It was noted that whilst the adam optimizer had the highest accuracy, the nadam optimizer has a higher validation accuracy, which would suggest a change in choice to the preliminary report.

In [9]:
# compile model, improving accuracy
base_model.compile(optimizer = "Nadam", loss = "categorical_crossentropy", metrics = ["accuracy"])
# train model, validating on test set
history = base_model.fit(augmented_images, one_hot_aug_train_labels, epochs = 10, validation_data = (test_images, one_hot_test_labels))

Epoch 1/10
368/368 [==============================] - 282s 760ms/step - loss: 0.8070 - accuracy: 0.6009 - val_loss: 1.1220 - val_accuracy: 0.5391
Epoch 2/10
368/368 [==============================] - 276s 750ms/step - loss: 0.5557 - accuracy: 0.7079 - val_loss: 1.1438 - val_accuracy: 0.5119
Epoch 3/10
368/368 [==============================] - 277s 753ms/step - loss: 0.4355 - accuracy: 0.7987 - val_loss: 3.4655 - val_accuracy: 0.5527
Epoch 4/10
368/368 [==============================] - 277s 752ms/step - loss: 0.2964 - accuracy: 0.8842 - val_loss: 2.0180 - val_accuracy: 0.5255
Epoch 5/10
368/368 [==============================] - 278s 755ms/step - loss: 0.1775 - accuracy: 0.9424 - val_loss: 2.5056 - val_accuracy: 0.5612
Epoch 6/10
368/368 [==============================] - 277s 753ms/step - loss: 0.1082 - accuracy: 0.9714 - val_loss: 2.5110 - val_accuracy: 0.5595
Epoch 7/10
368/368 [==============================] - 278s 755ms/step - loss: 0.0760 - accuracy: 0.9852 - val_loss: 4.2846 -

### Improving with additional layers

In [12]:
twoLayer_model = Sequential()

# first convolutional layer
twoLayer_model.add(Conv2D(32, (3, 3), activation = "relu", input_shape = (224, 224, 1)))
twoLayer_model.add(MaxPooling2D((2, 2)))

# second convolutional layer
twoLayer_model.add(Conv2D(64, (3, 3), activation = "relu"))
twoLayer_model.add(MaxPooling2D((2, 2)))

# unrolling output to 1D
twoLayer_model.add(Flatten())
twoLayer_model.add(Dense(128, activation = "relu"))
# using dropout for regularisation (reduces overfitting)
twoLayer_model.add(Dropout(0.5))
# output layer with softmax
twoLayer_model.add(Dense(2, activation = "softmax"))

In [13]:
# compile model, improving accuracy
twoLayer_model.compile(optimizer = "Nadam", loss = "categorical_crossentropy", metrics = ["accuracy"])
# train model, validating on test set
history = twoLayer_model.fit(augmented_images, one_hot_aug_train_labels, epochs = 10, validation_data = (test_images, one_hot_test_labels))

Epoch 1/10
368/368 [==============================] - 301s 815ms/step - loss: 0.6831 - accuracy: 0.5898 - val_loss: 0.9790 - val_accuracy: 0.5391
Epoch 2/10
368/368 [==============================] - 303s 823ms/step - loss: 0.5776 - accuracy: 0.6810 - val_loss: 2.0178 - val_accuracy: 0.5170
Epoch 3/10
368/368 [==============================] - 302s 820ms/step - loss: 0.4720 - accuracy: 0.7631 - val_loss: 2.9918 - val_accuracy: 0.5493
Epoch 4/10
368/368 [==============================] - 303s 824ms/step - loss: 0.3502 - accuracy: 0.8405 - val_loss: 3.0136 - val_accuracy: 0.5578
Epoch 5/10
368/368 [==============================] - 410s 1s/step - loss: 0.2529 - accuracy: 0.8965 - val_loss: 4.1739 - val_accuracy: 0.5391
Epoch 6/10
368/368 [==============================] - 419s 1s/step - loss: 0.1959 - accuracy: 0.9236 - val_loss: 7.5749 - val_accuracy: 0.5527
Epoch 7/10
368/368 [==============================] - 417s 1s/step - loss: 0.1680 - accuracy: 0.9401 - val_loss: 5.6220 - val_accu

In [16]:
threeLayer_model = Sequential()

# first convolutional layer
threeLayer_model.add(Conv2D(32, (3, 3), activation = "relu", input_shape = (224, 224, 1)))
threeLayer_model.add(MaxPooling2D((2, 2)))

# second convolutional layer
threeLayer_model.add(Conv2D(64, (3, 3), activation = "relu"))
threeLayer_model.add(MaxPooling2D((2, 2)))

# third convolutional layer
threeLayer_model.add(Conv2D(128, (3, 3), activation = "relu"))
threeLayer_model.add(MaxPooling2D((2, 2)))

# unrolling output to 1D
threeLayer_model.add(Flatten())
threeLayer_model.add(Dense(128, activation = "relu"))
# using dropout for regularisation (reduces overfitting)
threeLayer_model.add(Dropout(0.5))
# output layer with softmax
threeLayer_model.add(Dense(2, activation = "softmax"))

In [17]:
# compile model, improving accuracy
threeLayer_model.compile(optimizer = "Nadam", loss = "categorical_crossentropy", metrics = ["accuracy"])
# train model, validating on test set
history = threeLayer_model.fit(augmented_images, one_hot_aug_train_labels, epochs = 10, validation_data = (test_images, one_hot_test_labels))

Epoch 1/10
368/368 [==============================] - 532s 1s/step - loss: 0.6680 - accuracy: 0.5797 - val_loss: 1.4812 - val_accuracy: 0.5476
Epoch 2/10
368/368 [==============================] - 532s 1s/step - loss: 0.6202 - accuracy: 0.6213 - val_loss: 1.1769 - val_accuracy: 0.5578
Epoch 3/10
368/368 [==============================] - 531s 1s/step - loss: 0.5636 - accuracy: 0.6840 - val_loss: 1.1940 - val_accuracy: 0.5119
Epoch 4/10
368/368 [==============================] - 555s 2s/step - loss: 0.4830 - accuracy: 0.7457 - val_loss: 3.0069 - val_accuracy: 0.5578
Epoch 5/10
368/368 [==============================] - 462s 1s/step - loss: 0.3986 - accuracy: 0.8018 - val_loss: 2.0103 - val_accuracy: 0.5578
Epoch 6/10
368/368 [==============================] - 386s 1s/step - loss: 0.3171 - accuracy: 0.8513 - val_loss: 2.2953 - val_accuracy: 0.4983
Epoch 7/10
368/368 [==============================] - 2796s 8s/step - loss: 0.2646 - accuracy: 0.8800 - val_loss: 4.1996 - val_accuracy: 0.557